# MNIST Model Blind -- Attempt 6

Goal: To get a learner up and running using pytorch nets on the MNIST_Sample data, in the way that it's done in the book (Chapter 4). This is the last blind exercise I'll be doing for this chapter. Time to move on.

### Copied Data Piece

In [1]:
from fastai.data.all import untar_data, URLs
from pathlib import Path
from PIL import Image
import torch
from numpy import *

path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
seven_tensors = [torch.as_tensor(array(Image.open(o))) for o in sevens]
three_tensors = [torch.as_tensor(array(Image.open(o))) for o in threes]

In [2]:
stacked_sevens = torch.stack(seven_tensors).float() / 255
stacked_threes = torch.stack(three_tensors).float() / 255
stacked_sevens.shape, stacked_threes.shape

(torch.Size([6265, 28, 28]), torch.Size([6131, 28, 28]))

In [5]:
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_y = torch.as_tensor(
    array([1]*len(threes) + [0]*len(sevens))
).unsqueeze(1)
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [6]:
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

(torch.Size([784]), tensor([1]))

In [7]:
valid_3_tens = torch.stack([torch.as_tensor(array(Image.open(o))) 
                            for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([torch.as_tensor(array(Image.open(o))) 
                            for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [8]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = torch.as_tensor(array([1]*len(valid_3_tens) + [0]*len(valid_7_tens))).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

### Starting Here Without Reference

In [26]:
from fastai.data.load import DataLoader
from fastai.basics import DataLoaders

train_dl = DataLoader(dset, bs=256)
valid_dl = DataLoader(valid_dset, bs=256)
train_dl.one_batch()[0].shape, train_dl.one_batch()[1].shape

(torch.Size([256, 784]), torch.Size([256, 1]))

In [15]:
dls = DataLoaders(train_dl, valid_dl)

In [41]:
from torch import nn

model = nn.Sequential(
    nn.Linear(28*28, 1),
    nn.Sigmoid()
)

In [42]:
def loss(preds, tars): return torch.where(tars==1, 1-preds, preds).mean()

In [43]:
def accuracy(preds, tars): return ((preds > 0.5) == tars).float().mean()

In [44]:
from fastai.basics import Learner
from fastai.optimizer import SGD

learn = Learner(dls, model, loss_func=loss, metrics=accuracy, opt_func=SGD)

In [45]:
learn.fit(10)

[0, 0.5011638402938843, 0.49241912364959717, 0.5912659764289856, '00:00']
[1, 0.4871678948402405, 0.4717788100242615, 0.7502453327178955, '00:00']
[2, 0.47107040882110596, 0.4514162540435791, 0.8640824556350708, '00:00']
[3, 0.45375362038612366, 0.4315725862979889, 0.9214916825294495, '00:00']
[4, 0.43596887588500977, 0.4124457538127899, 0.9425907731056213, '00:00']
[5, 0.41825178265571594, 0.3941809833049774, 0.9499509334564209, '00:00']
[6, 0.4009486138820648, 0.3768712282180786, 0.9568204283714294, '00:00']
[7, 0.38427141308784485, 0.36056438088417053, 0.9573110938072205, '00:00']
[8, 0.36834585666656494, 0.3452725112438202, 0.9592738151550293, '00:00']
[9, 0.3532417416572571, 0.3309812545776367, 0.9602551460266113, '00:00']


### Awesome!

That only took about 15 minutes! I hit one error at the end, but figured it out without reference.

Very happy with the progress made here. I was definitely hitting diminishing returns with some of this, but it's also really helpful to know parts of the API without thinking or looking them up.

Moving on...